# Import libraries

In [ ]:
%matplotlib inline
#import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T  
from scipy.stats import norm
from itertools import combinations
from sklearn.utils import shuffle
from numpy import array
from scipy.io import loadmat

# Set up display

In [ ]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

# Deep Q-Network

In [ ]:
class DQN1(nn.Module):

    def __init__(self,m):
        super().__init__()
            
        self.fc1 = nn.Linear(in_features=m, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=256)
        self.fc4 = nn.Linear(in_features=256, out_features=128)
        self.fc5 = nn.Linear(in_features=128, out_features=64)
        self.out = nn.Linear(in_features=64, out_features=7)
    def forward(self, t):
        
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = F.relu(self.fc3(t))
        t = F.relu(self.fc4(t))
        t = F.relu(self.fc5(t))
        t = self.out(t)
        return t
    

# Experience class

In [ ]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

# Replay Memory

In [ ]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience)
        else:
            self.memory[self.push_count % self.capacity] = experience
        self.push_count += 1
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    
    def can_provide_sample(self, batch_size):
        return len(self.memory) >= batch_size

# Epsilon Greedy Strategy

In [ ]:
class EpsilonGreedyStrategy():
    def __init__(self, start, end, decay):
        self.start = start
        self.end = end
        self.decay = decay
        
    def get_exploration_rate(self, current_step):
        return self.end + (self.start - self.end) * \
            math.exp(-1. * current_step * self.decay)

# Reinforcement Learning Agent

In [ ]:
class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device
        
    def select_action(self, state, policy_net,k,point):
        rate = self.strategy.get_exploration_rate(self.current_step) 
        self.current_step += k
        c = 0
        
        if point == 0 :
            if rate > random.random():
                return torch.tensor([[random.randrange(self.num_actions)]], device=device, dtype=torch.long),c,rate
            else:
                c = c + 1;
                with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
                    return policy_net(state).argmax().view(1, 1),c,rate
                #return policy_net(state).argmax().item()
        else:
            c = c + 1;
            with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
                return policy_net(state).argmax().view(1, 1),c,rate
 

# Take action

In [ ]:
#def take_action(action1,number_action,state,h0,nTx,Es_N0,act):     
def take_action(action1,state,h0,nTx,Es_N0,S): 
    
    counter = 0
    i = 0
 #   th = pow(10,-3)
    device = torch.device("cpu" if torch.cuda.is_available() else "cpu") 
    action = np.array([2,4,8,16,32,64,128])
    action = torch.tensor([action], device=device) 
    ModulationOrder = action[0,action1]

    SNR = Es_N0*pow(10,(h0-30)/10)
    
    if ModulationOrder == 2:
        i = 2
        BER = 1-(norm.cdf(math.sqrt(2*SNR)))
    elif ModulationOrder == 4:
        i = 4
        BER = (2/2)*(1-(1/math.sqrt(4)))*(1-norm.cdf(math.sqrt((3*SNR)/(4-1))))
    elif ModulationOrder == 8:
        i = 8
        BER = (2/3)*(1-(1/math.sqrt(8)))*(1-norm.cdf(math.sqrt((3*SNR)/(8-1))))
    elif ModulationOrder == 16:
        i = 16
        BER = (2/4)*(1-(1/math.sqrt(16)))*(1-norm.cdf(math.sqrt((3*SNR)/(16-1))))
    elif ModulationOrder == 32:
        i = 32
        BER = (2/5)*(1-(1/math.sqrt(32)))*(1-norm.cdf(math.sqrt((3*SNR)/(32-1))))
    elif ModulationOrder == 64:
        i = 64
        BER = (2/6)*(1-(1/math.sqrt(64)))*(1-norm.cdf(math.sqrt((3*SNR)/(64-1))))
    elif ModulationOrder == 128:
        i = 128
        BER = (2/7)*(1-(1/math.sqrt(128)))*(1-norm.cdf(math.sqrt((3*SNR)/(128-1))))

    
    R = S*math.log(ModulationOrder,2) * ((1-math.log(ModulationOrder,2)*BER)**S)

    reward1 = R

    return torch.tensor([reward1], device=device),counter,BER,ModulationOrder,SNR

# Get state

# Utility functions

In [ ]:
def plot(values, moving_avg_period):
    plt.figure(2)
    plt.clf()        
    plt.title('Training...')
    plt.xlabel('Time step')
    plt.ylabel('Throughput(bits/s)')
    plt.plot(values)
    moving_avg = get_moving_average(moving_avg_period, values)
    plt.plot(moving_avg)    
    plt.pause(0.001)
    print("Time step", len(values), "\n", \
            moving_avg_period, "Time step moving avg:", moving_avg[-1])
    if is_ipython: display.clear_output(wait=True)
def get_moving_average(period, values):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1) \
            .mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()


# Tensor processing

In [ ]:
def extract_tensors(experiences):
    # Convert batch of Experiences to Experience of batches
    batch = Experience(*zip(*experiences))

    t1 = torch.cat(batch.state)
    t2 = torch.cat(batch.action)
    t3 = torch.cat(batch.reward)
    t4 = torch.cat(batch.next_state)

    return (t1,t2,t3,t4)

# calculating Q_values

In [ ]:
class QValues():
    device = torch.device("cpu" if torch.cuda.is_available() else "cpu")
    @staticmethod
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1 ,index=actions)
    @staticmethod        
    def get_next(target_net, next_states):                
        final_state_locations = next_states.flatten(start_dim=1) \
            .max(dim=1)[0].eq(0).type(torch.bool)
        non_final_state_locations = (final_state_locations == False)
        non_final_states = next_states[non_final_state_locations]
        batch_size = next_states.shape[0]
        values = torch.zeros(batch_size).to(QValues.device)
        values[non_final_state_locations] = target_net(non_final_states).max(dim=1)[0].detach()
        return values
    

# Main

In [ ]:
from collections import OrderedDict
batch_size = 1000
gamma = 0.001
eps_start = 1
eps_end = 0.0001
eps_decay = 0.0001
eps_th = 0.0002
target_update = 100
memory_size1 = 50000
lr = 0.003
num_episodes = 1000
num_actions_available1 =7 


nRx = 1
nTx = 1

mm=10
mm_state=np.zeros((mm,nRx,nTx),np.float64)
mm_state2=np.zeros((mm,nRx,nTx),np.float64)

SNRdB = np.arange(110,111)

epsilon = 1
epsilon1 = 1

device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

NN = 10000
S = 1000
point = 0

reward_final = []
reward_final1 = []
SNR_final = []
BER_final = []
SER_final = []
rate_final = []
rate1_final = []
rate2_final = []
ModulationOrder_final = []
ModulationOrder_final123 = []

annots = loadmat('R999_new.mat')
h_first = annots['R']
print((h_first.shape))
h1 = h_first[mm-1:np.shape(h_first)[0]]
mm_state[1:mm] = np.reshape(np.fliplr(h_first[0:mm-1]),(mm-1,1,1))


episode_durations = []
reward_f_con =[]
reward_f = []
for ii in range (np.shape(SNRdB)[0]):
    Es_N0 = pow(10,(SNRdB[ii]/10))
    print(ii)
    point = 0
    epsilon = 1
    episode_durations=[]

    reward_matrix = []
    BER_matrix = []
    SER_matrix = []
    SNR_matrix = []
    rate_rate = []
    rate_rate1 = []
    rate_rate2 = []
    reward_matrix_con=[]
    reward_matrix = []
    NETW = []
    ModulationOrder_matrix = []
    
    strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
    agent = Agent(strategy,num_actions_available1 , device)
    memory1 = ReplayMemory(memory_size1)

    policy_net1 = DQN1(mm).to(device)    
    target_net1 = DQN1(mm).to(device)

    target_net1.load_state_dict(policy_net1.state_dict())
    target_net1.eval()
    optimizer1 = optim.Adam(params=policy_net1.parameters(), lr=lr)
    
    for episode in range(0,1):

        strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
        agent = Agent(strategy,num_actions_available1 , device)
        memory1 = ReplayMemory(memory_size1)
        
        target_net1.load_state_dict(policy_net1.state_dict())
        target_net1.eval()

        optimizer1 = optim.Adam(params=policy_net1.parameters(), lr=lr)

        mm_state[1:mm] = np.reshape(np.fliplr(h_first[0:mm-1]),(mm-1,1,1))
        i = 0
        k = 1
        h0 = h1[i]
        state0 = h1[i]

        mm_state[0]=state0
        state =torch.tensor([mm_state],dtype=torch.float32, device=device)
        point = 0
        epsilon = 1
            
        for timestep in range(np.shape(h1)[0]-1):

            i = i+1
            action,c,epsilon = agent.select_action(state,policy_net1,k,point)
            h0 = h1[i]
            next_state0 = h1[i] 
            mm_state = np.roll(mm_state,1,axis=0)
            mm_state[0] = next_state0
            next_state = torch.tensor([mm_state],dtype=torch.float32, device=device)
            
            reward,counter,BER,ModulationOrder,SNRR = take_action(action,next_state,h0,nTx,Es_N0,S)
            memory1.push(Experience(state, action, next_state, reward))
            state = next_state
            
            reward_matrix.append(reward)

            
            if epsilon < eps_th and point == 0:
                point = point+1
        

            if point == 0:

                if memory1.can_provide_sample(batch_size):
                    experiences = memory1.sample(batch_size)
                    states, actions, rewards, next_states = extract_tensors(experiences)
                    current_q_values = QValues.get_current(policy_net1, states, actions)
                    next_q_values = QValues.get_next(target_net1, next_states)
                    target_q_values = (next_q_values * gamma) + rewards

                    loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
                    optimizer1.zero_grad()
                    loss.backward()
                    optimizer1.step()
                
                if timestep % target_update == 0:

                    target_net1.load_state_dict(policy_net1.state_dict())
                    
        reward_final.append(reward_matrix)

        np.savez("reward_dete2_snr22_112", reward_final)
